### Imports

In [1]:
from datetime import datetime
import time

from contracts_lib_py.account import Account
from common_utils_py.agreements.service_types import ServiceTypesIndices


from nevermined_sdk_py import Config, Nevermined
from nevermined_sdk_py.nevermined.keeper import NeverminedKeeper as Keeper

from workshop import utils

CONSUMER_ADDRESS = "0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e"
CONSUMER_PASSWORD = "node0"
CONSUMER_KEYFILE = "../resources/accounts/consumer.json"
CONFIG_FILE = "../config.ini"

PROVIDER_ADDRESS = "0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0"
ASSET_COMPUTE_DID_1 = "did:nv:f86b0e3bc9172159923dd08e87950180bc4b7a71405b7734460f6ce73724e65b"
ASSET_COMPUTE_DID_2 = "did:nv:65570ec086f307e32e5da12e6abeeb153c14bce8c1e82c35c5fc3821d211968d"
ASSET_COORDINATOR_DID = "did:nv:4efddfe44b3f6cd7e6a6847ec492ab0baf72bbda1507eede6ead543f16856935"

### Setup Nevermined and accounts

In [2]:
nevermined = Nevermined(Config(CONFIG_FILE))
consumer_account = Account(CONSUMER_ADDRESS, CONSUMER_PASSWORD, CONSUMER_KEYFILE)

nevermined.accounts.request_tokens(consumer_account, 10)


True

### Publish algorithm

In [3]:
metadata_algorithm = utils.metadata_algorithm_template()
utils.print_json(metadata_algorithm)

metadata_algorithm["main"]["dateCreated"] = utils.date_now()
metadata_algorithm["main"]["files"][0]["url"] = "https://github.com/nevermined-io/hellodecentralization/raw/main/notebooks/federated_fraud_demo.ipynb"

metadata_algorithm["main"]["algorithm"]["entyrypoint"] = (
    "pip install jupyter pandas papermill scikit-learn xain-sdk && "
    "papermill --stdout-file - federated_fraud_demo.ipynb $NEVERMINED_OUTPUTS_PATH/federated_fraud_demo_output.ipynb"
)

metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["image"] = "python"
metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["tag"] = "3.8-slim-buster"


ddo_algorithm = nevermined.assets.create(metadata_algorithm, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm.did}")

{
  "main": {
    "name": "Fraud Demo Algorithm",
    "dateCreated": null,
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "files": [
      {
        "index": 0,
        "contentType": "text/text",
        "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
        "checksumType": "MD5",
        "contentLength": "12057507",
        "url": null
      }
    ],
    "type": "algorithm",
    "algorithm": {
      "language": "python",
      "format": "py",
      "version": "0.1.0",
      "entrypoint": null,
      "requirements": {
        "container": {
          "image": null,
          "tag": null,
          "checksum": "sha256:53ad3a03b2fb240b6c494339821e6638cd44c989bcf26ec4d51a6a52f7518c1d"
        }
      }
    }
  }
}
Algorithm DID: did:nv:8ac28a3ec0a3dd9f215e769433a87148c4fcfe6ad851690b303a07eb0f21873f


### Publish the workflows:

- Two for the compute to the data assets
- One for the coordinator service

In [4]:
# Compute asset 1
metadata_workflow1 = utils.metadata_workflow_template()
utils.print_json(metadata_workflow1)

metadata_workflow1["main"]["dateCreated"] = utils.date_now()
metadata_workflow1["main"]["workflow"]["stages"][0]["input"][0]["id"] = ASSET_COMPUTE_DID_1
metadata_workflow1["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

ddo_workflow1 = nevermined.assets.create(metadata_workflow1, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow1.did}")

# Compute asset 2
metadata_workflow2 = utils.metadata_workflow_template()
metadata_workflow2["main"]["dateCreated"] = utils.date_now()
metadata_workflow2["main"]["workflow"]["stages"][0]["input"][0]["id"] = ASSET_COMPUTE_DID_2
metadata_workflow2["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

ddo_workflow2 = nevermined.assets.create(metadata_workflow2, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow2.did}")

# Coordinator service
metadata_workflow_coordinator = utils.metadata_workflow_coordinator()
metadata_workflow_coordinator["main"]["dateCreated"] = utils.date_now()
metadata_workflow_coordinator["main"]["datePublished"] = utils.date_now()

ddo_workflow_coordinator = nevermined.assets.create(metadata_workflow_coordinator, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow_coordinator.did}")

{
  "main": {
    "name": "Fraud Demo Workflow",
    "dateCreated": null,
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [
            {
              "index": 0,
              "id": null
            }
          ],
          "transformation": {
            "id": null
          }
        }
      ]
    }
  }
}
Workflow DID: did:nv:31b1e6ebf41a45ce1cfb22854fd5cb9864909f1c1479909c38df6c186f6f7247
Workflow DID: did:nv:935085f55474da2d70b7308bc4a85bee672fa827bea8e06a303e95101c6255c4
Workflow DID: did:nv:b95a63bf19517f295be4248ba8ce977d7ea41dc455c22955ad475bd9aa23a629


### Order computations:

- One for each data asset
- One for the coordinator service

In between we want for the blockchain to process the request.

In [5]:
keeper = Keeper.get_instance()

service_agreement_id1 = nevermined.assets.order(ASSET_COMPUTE_DID_1, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id1}")
utils.wait_for_event(keeper, service_agreement_id1)

service_agreement_id2 = nevermined.assets.order(ASSET_COMPUTE_DID_2, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id2}")
utils.wait_for_event(keeper, service_agreement_id2)

service_agreement_id_coordinator = nevermined.assets.order(ASSET_COORDINATOR_DID, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id_coordinator}")
utils.wait_for_event(keeper, service_agreement_id_coordinator)


Service Agreement ID: 0x1dcd6bbdc42f4086a46d616085dcac0f5f80752419a742558f88e5672ee489b3
Service Agreement ID: 0x0586c17cb28c40a3953fc41f521ac7adaec9eaeecf0f422b9db7c7bd4d0a7401
Service Agreement ID: 0x1dbad5160c4a400ca71e811a0e7d047d0be9e311f6cd42d5a9fb936e0a571afc


### Execute workflows

This orders the Nevermined to start the execution of the workflows:
- One for each data asset
- One for the coordinator service

In [6]:
execution_id1 = nevermined.assets.execute(
    service_agreement_id1,
    ASSET_COMPUTE_DID_1,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow1.did
)
print(f"Execution ID: {execution_id1}")

execution_id2 = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_COMPUTE_DID_2,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow2.did
)
print(f"Execution ID: {execution_id2}")

execution_id_coordinator = nevermined.assets.execute(
    service_agreement_id_coordinator,
    ASSET_COORDINATOR_DID,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow_coordinator.did
)
print(f"Execution ID: {execution_id_coordinator}")

Execution ID: nevermined-compute-pmqlj
Execution ID: nevermined-compute-7ktr4
Execution ID: nevermined-compute-cljtt


### Wait for compute jobs to finish

In [7]:
succeeded = set()
failed = set()
outputs = set()
while len(succeeded) < 3:
    for job_id in [execution_id1, execution_id2, execution_id_coordinator]:
        result = nevermined.assets.compute_status(service_agreement_id_coordinator, job_id, consumer_account)
        status = result["status"]
        print(f"{job_id}: {status}")

        if status == "Succeeded":
            succeeded.add(job_id)
            outputs.add(result["did"])
        elif status == "Failed":
            raise Exception("Some jobs failed")
    print()
    time.sleep(10)

for did in outputs:
    print(f"Output DID: {did}")

ValueError: The compute API was not able to return the logs. b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

### Download the results

In [ ]:
for did in outputs:
    print(f"Downloading: {did}")
    nevermined.assets.download(did, ServiceTypesIndices.DEFAULT_ACCESS_INDEX, consumer_account, "./")